In [45]:
# import libraries 
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math 
from scipy import stats

In [46]:
# import stocks and API token
stocks = pd.read_csv('..\data\sp_500_stocks.csv')

from secrets import IEX_CLOUD_API_TOKEN

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\sp_500_stocks.csv'

In [ ]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   

In [ ]:
# creates chunks of 100 from the stock list and take every group of stocks in the symbol_groups to be a comma separated string
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

In [ ]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# metrics for value investing
# D/E Ratio - proportion of equity to debt a company is using to finance its assets.
de_ratio = data[symbol]['advanced-stats']['debtToEquity']

# P/E Ratio - what the market is willing to pay for a stock based on past or future earnings.
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio - measures whether a stock is over/undervalued by comparing the net value (assets - liabilities) to its market capitalization.
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio - compares stock price to its revenues.
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

#PEG Ratio - measures the relationship between the price/earnings ratio and earnings growth
peg_ratio = data[symbol]['advanced-stats']['pegRatio']

# EV/EBITDA - comparing the value of a company, debt included, to the company’s cash earnings less non-cash expenses.
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP - how many dollars of enterprise value are generated for every dollar of gross profit earned.
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [ ]:
# Create columns
my_columns = [
    'Ticker', 
    'Price', 
    'Number of Shares to Buy',
    'Price-to-Earnings-to-Growth Ratio',
    'PEG Percentile',
    'Debt-to-Equity Ratio',
    'DE Percentile',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

In [ ]:
# Create an empty dataframe with the specified columns
df = pd.DataFrame(columns = my_columns)

In [ ]:
# loop data from batch API call and applying it to the df
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        ev = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gp = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = ev/ebitda
        except:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gross_profit = ev/gp
        except:
            ev_to_gross_profit = np.NaN
        
        df = df.append(
            pd.Series(
                [ 
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['advanced-stats']['pegRatio'],
                'N/A',
                data[symbol]['advanced-stats']['debtToEquity'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )

df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings-to-Growth Ratio,PEG Percentile,Debt-to-Equity Ratio,DE Percentile,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,158.50,N/A,1.107783,N/A,2.179795,N/A,49.18,N/A,9.73,N/A,8.12,N/A,32.147445,N/A,14.903421,N/A,N/A
1,AAL,19.09,N/A,-0.058659,N/A,-9.50192,N/A,-3.79,N/A,-1.62,N/A,0.506,N/A,-7.390757,N/A,1.511644,N/A,N/A
2,AAP,240.50,N/A,0.540226,N/A,3.772645,N/A,24.58,N/A,4.57,N/A,1.37,N/A,13.540927,N/A,3.026151,N/A,N/A
3,AAPL,178.58,N/A,0.108823,N/A,5.733869,N/A,16.2,N/A,23.32,N/A,4.07,N/A,12.606144,N/A,9.681496,N/A,N/A
4,ABBV,139.46,N/A,-3.767813,N/A,11.128528,N/A,33.22,N/A,18.2,N/A,4.46,N/A,11.762162,N/A,8.334230,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.37,N/A,0.50357,N/A,-0.843694,N/A,26.8,N/A,-5.18,N/A,6.2,N/A,21.728958,N/A,10.583395,N/A,N/A
501,ZBH,130.85,N/A,0.051751,N/A,1.954272,N/A,33.37,N/A,2.1,N/A,3.43,N/A,17.754721,N/A,5.846700,N/A,N/A
502,ZBRA,593.02,N/A,0.468827,N/A,2.133692,N/A,38.69,N/A,11.16,N/A,5.93,N/A,27.646981,N/A,12.278042,N/A,N/A
503,ZION,63.73,N/A,0.016908,N/A,11.380188,N/A,6.05,N/A,1.37,N/A,2.9,N/A,4.697276,N/A,2.490942,N/A,N/A


In [ ]:
# Remove None
df.dropna(inplace = True)
df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings-to-Growth Ratio,PEG Percentile,Debt-to-Equity Ratio,DE Percentile,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,158.50,N/A,1.107783,N/A,2.179795,N/A,49.18,N/A,9.73,N/A,8.12,N/A,32.147445,N/A,14.903421,N/A,N/A
1,AAL,19.09,N/A,-0.058659,N/A,-9.50192,N/A,-3.79,N/A,-1.62,N/A,0.506,N/A,-7.390757,N/A,1.511644,N/A,N/A
2,AAP,240.50,N/A,0.540226,N/A,3.772645,N/A,24.58,N/A,4.57,N/A,1.37,N/A,13.540927,N/A,3.026151,N/A,N/A
3,AAPL,178.58,N/A,0.108823,N/A,5.733869,N/A,16.2,N/A,23.32,N/A,4.07,N/A,12.606144,N/A,9.681496,N/A,N/A
4,ABBV,139.46,N/A,-3.767813,N/A,11.128528,N/A,33.22,N/A,18.2,N/A,4.46,N/A,11.762162,N/A,8.334230,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.37,N/A,0.50357,N/A,-0.843694,N/A,26.8,N/A,-5.18,N/A,6.2,N/A,21.728958,N/A,10.583395,N/A,N/A
501,ZBH,130.85,N/A,0.051751,N/A,1.954272,N/A,33.37,N/A,2.1,N/A,3.43,N/A,17.754721,N/A,5.846700,N/A,N/A
502,ZBRA,593.02,N/A,0.468827,N/A,2.133692,N/A,38.69,N/A,11.16,N/A,5.93,N/A,27.646981,N/A,12.278042,N/A,N/A
503,ZION,63.73,N/A,0.016908,N/A,11.380188,N/A,6.05,N/A,1.37,N/A,2.9,N/A,4.697276,N/A,2.490942,N/A,N/A


In [ ]:
from scipy.stats import percentileofscore as score

# Calculating value score percentiles for each stock
metrics = {
    'Price-to-Earnings-to-Growth Ratio': 'PEG Percentile',
    'Debt-to-Equity Ratio': 'DE Percentile',
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in df.index:
        df.loc[row, metrics[metric]] = score(df[metric], df.loc[row, metric])/100

df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings-to-Growth Ratio,PEG Percentile,Debt-to-Equity Ratio,DE Percentile,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,158.50,N/A,1.107783,0.792608,2.179795,0.316222,49.18,0.829569,9.73,0.792608,8.12,0.825462,32.147445,0.882957,14.903421,0.841889,N/A
1,AAL,19.09,N/A,-0.058659,0.205339,-9.50192,0.039014,-3.79,0.053388,-1.62,0.055441,0.506,0.053388,-7.390757,0.022587,1.511644,0.069815,N/A
2,AAP,240.50,N/A,0.540226,0.657084,3.772645,0.673511,24.58,0.517454,4.57,0.606776,1.37,0.201232,13.540927,0.462012,3.026151,0.172485,N/A
3,AAPL,178.58,N/A,0.108823,0.414784,5.733869,0.823409,16.2,0.308008,23.32,0.936345,4.07,0.613963,12.606144,0.422998,9.681496,0.652977,N/A
4,ABBV,139.46,N/A,-3.767813,0.041068,11.128528,0.917864,33.22,0.669405,18.2,0.911704,4.46,0.64271,11.762162,0.390144,8.334230,0.577002,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.37,N/A,0.50357,0.638604,-0.843694,0.053388,26.8,0.564682,-5.18,0.051335,6.2,0.75462,21.728958,0.726899,10.583395,0.696099,N/A
501,ZBH,130.85,N/A,0.051751,0.340862,1.954272,0.223819,33.37,0.671458,2.1,0.311088,3.43,0.532854,17.754721,0.620123,5.846700,0.400411,N/A
502,ZBRA,593.02,N/A,0.468827,0.62423,2.133692,0.297741,38.69,0.743326,11.16,0.825462,5.93,0.73614,27.646981,0.819302,12.278042,0.767967,N/A
503,ZION,63.73,N/A,0.016908,0.258727,11.380188,0.930185,6.05,0.094456,1.37,0.157084,2.9,0.470226,4.697276,0.069815,2.490942,0.129363,N/A


In [ ]:
# Calculating the robust value score
# RV score is the arithmetic mean of all the percentile scores

from statistics import mean 

for row in df.index:
    value_percentiles = []

    for metric in metrics.keys():
        value_percentiles.append(df.loc[row, metrics[metric]])
    df.loc[row, 'RV Score'] = mean(value_percentiles)

df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings-to-Growth Ratio,PEG Percentile,Debt-to-Equity Ratio,DE Percentile,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,158.50,N/A,1.107783,0.792608,2.179795,0.316222,49.18,0.829569,9.73,0.792608,8.12,0.825462,32.147445,0.882957,14.903421,0.841889,0.754473
1,AAL,19.09,N/A,-0.058659,0.205339,-9.50192,0.039014,-3.79,0.053388,-1.62,0.055441,0.506,0.053388,-7.390757,0.022587,1.511644,0.069815,0.071282
2,AAP,240.50,N/A,0.540226,0.657084,3.772645,0.673511,24.58,0.517454,4.57,0.606776,1.37,0.201232,13.540927,0.462012,3.026151,0.172485,0.470079
3,AAPL,178.58,N/A,0.108823,0.414784,5.733869,0.823409,16.2,0.308008,23.32,0.936345,4.07,0.613963,12.606144,0.422998,9.681496,0.652977,0.596069
4,ABBV,139.46,N/A,-3.767813,0.041068,11.128528,0.917864,33.22,0.669405,18.2,0.911704,4.46,0.64271,11.762162,0.390144,8.334230,0.577002,0.592842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.37,N/A,0.50357,0.638604,-0.843694,0.053388,26.8,0.564682,-5.18,0.051335,6.2,0.75462,21.728958,0.726899,10.583395,0.696099,0.497947
501,ZBH,130.85,N/A,0.051751,0.340862,1.954272,0.223819,33.37,0.671458,2.1,0.311088,3.43,0.532854,17.754721,0.620123,5.846700,0.400411,0.442945
502,ZBRA,593.02,N/A,0.468827,0.62423,2.133692,0.297741,38.69,0.743326,11.16,0.825462,5.93,0.73614,27.646981,0.819302,12.278042,0.767967,0.687738
503,ZION,63.73,N/A,0.016908,0.258727,11.380188,0.930185,6.05,0.094456,1.37,0.157084,2.9,0.470226,4.697276,0.069815,2.490942,0.129363,0.301408


In [ ]:
# Top 100 best value stocks by RV Score
df.sort_values('RV Score', ascending = True, inplace = True)
df = df[:100]
df.reset_index(drop = True, inplace = True)
df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings-to-Growth Ratio,PEG Percentile,Debt-to-Equity Ratio,DE Percentile,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,19.09,N/A,-0.058659,0.205339,-9.50192,0.039014,-3.79,0.053388,-1.62,0.055441,0.506,0.053388,-7.390757,0.022587,1.511644,0.069815,0.071282
1,HPQ,38.83,N/A,0.009691,0.232033,-24.5,0.022587,3.62,0.069815,-12.92,0.036961,0.336,0.01848,4.115003,0.055441,1.755325,0.088296,0.074802
2,MCK,247.86,N/A,0.023215,0.277207,-114.459202,0.004107,-7.92,0.049281,-66.69,0.020534,0.1508,0.008214,9.355300,0.279261,3.232812,0.186858,0.117923
3,DHI,105.23,N/A,0.029108,0.287474,1.623092,0.12115,4.8,0.079055,1.27,0.13963,0.6961,0.075975,3.611461,0.039014,2.464198,0.12731,0.12423
4,BA,205.52,N/A,0.170962,0.482546,-10.57714,0.034908,-14.75,0.043121,-8.32,0.047228,1.99,0.332649,-32.624330,0.00616,-1315.857668,0.002053,0.135524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DE,355.27,N/A,0.038953,0.318275,4.688786,0.767967,9.53,0.158111,3.01,0.451745,1.26,0.180698,6.360524,0.117043,3.745042,0.232033,0.317982
96,NWL,22.35,N/A,0.022404,0.271047,3.64467,0.655031,15.73,0.301848,2.29,0.335729,0.8708,0.10883,9.875070,0.295688,4.141228,0.26078,0.318422
97,PGR,102.68,N/A,-0.743464,0.151951,4.004312,0.704312,14.94,0.289528,3.33,0.489733,1.31,0.187885,11.007198,0.353183,1.280185,0.055441,0.318862
98,PXD,188.85,N/A,0.070087,0.36961,1.628129,0.125257,30.51,0.628337,2,0.301848,3.13,0.491786,7.364146,0.166324,2.897842,0.166324,0.321355


In [ ]:
# Calculate the number of shares to buy
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That is not a number! \n Try again:")
        portfolio_size = input('Enter the value of your portfolio:')

In [ ]:
portfolio_input()

In [ ]:
# Calculates the inputted portfolio amount and equal weight on all stocks by price
position_size = float(portfolio_size)/len(df.index)

for row in df.index:
    df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/df.loc[row, 'Price'])

df

C:\Users\dopey\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings-to-Growth Ratio,PEG Percentile,Debt-to-Equity Ratio,DE Percentile,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,19.09,5238,-0.058659,0.205339,-9.50192,0.039014,-3.79,0.053388,-1.62,0.055441,0.506,0.053388,-7.390757,0.022587,1.511644,0.069815,0.071282
1,HPQ,38.83,2575,0.009691,0.232033,-24.5,0.022587,3.62,0.069815,-12.92,0.036961,0.336,0.01848,4.115003,0.055441,1.755325,0.088296,0.074802
2,MCK,247.86,403,0.023215,0.277207,-114.459202,0.004107,-7.92,0.049281,-66.69,0.020534,0.1508,0.008214,9.355300,0.279261,3.232812,0.186858,0.117923
3,DHI,105.23,950,0.029108,0.287474,1.623092,0.12115,4.8,0.079055,1.27,0.13963,0.6961,0.075975,3.611461,0.039014,2.464198,0.12731,0.12423
4,BA,205.52,486,0.170962,0.482546,-10.57714,0.034908,-14.75,0.043121,-8.32,0.047228,1.99,0.332649,-32.624330,0.00616,-1315.857668,0.002053,0.135524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DE,355.27,281,0.038953,0.318275,4.688786,0.767967,9.53,0.158111,3.01,0.451745,1.26,0.180698,6.360524,0.117043,3.745042,0.232033,0.317982
96,NWL,22.35,4474,0.022404,0.271047,3.64467,0.655031,15.73,0.301848,2.29,0.335729,0.8708,0.10883,9.875070,0.295688,4.141228,0.26078,0.318422
97,PGR,102.68,973,-0.743464,0.151951,4.004312,0.704312,14.94,0.289528,3.33,0.489733,1.31,0.187885,11.007198,0.353183,1.280185,0.055441,0.318862
98,PXD,188.85,529,0.070087,0.36961,1.628129,0.125257,30.51,0.628337,2,0.301848,3.13,0.491786,7.364146,0.166324,2.897842,0.166324,0.321355


In [ ]:

# xlsxwriter library to create excel file
writer = pd.ExcelWriter('100_value_stock_strategy.xlsx', engine = 'xlsxwriter')

df.to_excel(writer, sheet_name = '100 Value Stock Strategy', index = False)

In [ ]:
# excel format
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
# apply formats to columns
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earnings-to-Growth Ratio', float_template],
    'E': ['PEG Percentile', percent_template],
    'F': ['Debt-to-Equity Ratio', float_template],
    'G': ['DE Percentile', percent_template],
    'H': ['Price-to-Earnings Ratio', float_template],
    'I': ['PE Percentile', percent_template],
    'J': ['Price-to-Book Ratio', float_template],
    'K': ['PB Percentile', percent_template],
    'L': ['Price-to-Sales Ratio', float_template],
    'M': ['PS Percentile', percent_template],
    'N': ['EV/EBITDA', float_template],
    'O': ['EV/EBITDA Percentile', percent_template],
    'P': ['EV/GP', float_template],
    'Q': ['EV/GP Percentile', percent_template],
    'R': ['RV Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['100 Value Stock Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['100 Value Stock Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [ ]:
# save excel output
writer.save()